# **A. Préparation: bibliothèques, données, preprocessing**

## 1. Importation des bibliothèques nécessaires.

In [1]:
import numpy as np
import pandas as pd
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings("ignore")
from preprocessing import preprocess_data
from outils import save_to_csv

## 2. Importation des données

In [2]:
dataset_train = pd.read_csv("../data/x_train.csv")
dataset_test = pd.read_csv("../data/x_test.csv")

In [3]:
dataset_train.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low


## 3. Définition des labels

Cette cellule définit deux dictionnaires pour mapper les catégories de niveaux d'eau des nappes phréatiques entre leur représentation en entiers (utilisée pour le traitement numérique) et leur représentation en chaînes de caractères (plus compréhensible pour les humains).

In [4]:
mapping_int_to_string = {
    0: 'Average',
    1: 'High',
    2: 'Low',
    3: 'Very High',
    4: 'Very Low'
}
mapping_string_to_int = {value: key for key, value in mapping_int_to_string.items()}

## 4.Séparation, transformation et prétraitement des données d'entraînement et de test.

### Description des variables:

Données d'entraînement (features et labels) :
- X_train : Features uniquement.
- y_train : Labels (catégories sous forme entière).
- row_index_train : Identifiants des lignes.

Données de test :
- X_test : Features uniquement.
- row_index_test : Identifiants des lignes.

Données combinées :
- all_features : Fusion des features des deux jeux (train + test).

Données prétraitées :
- preprocessed_features_all : Données prétraitées combinées.
- X_train_preprocessed : Partie entraînement des données prétraitées.
- X_test_preprocessed : Partie test des données prétraitées.

In [5]:
# Séparation des caractéristiques, des labels, et des indices pour le jeu d'entraînement
X_train, y_train, row_index_train = dataset_train.drop(columns=["piezo_groundwater_level_category","row_index"]), dataset_train["piezo_groundwater_level_category"].map(mapping_string_to_int), dataset_train["row_index"]

# Séparation des caractéristiques et des indices pour le jeu de test
X_test, row_index_test = dataset_test.drop(columns=["row_index"]), dataset_test["row_index"]

# Fusion des caractéristiques des jeux d'entraînement et de test pour prétraitement uniforme
all_features = pd.concat([X_train, X_test], axis=0)

# Prétraitement des données combinées
preprocessed_features_all = preprocess_data(all_features)

# Division des données prétraitées en entraînement et test
X_train_preprocessed = preprocessed_features_all[:len(X_train)]
X_test_preprocessed = preprocessed_features_all[len(X_train):]

Columns with mixed types: ['piezo_station_department_code', 'piezo_station_commune_code_insee', 'piezo_measure_nature_code', 'prelev_structure_code_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_structure_code_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_structure_code_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2']
No missing values in the dataset


A présent, on vérifie si le jeu de données combiné (preprocessed_features_all) contient des valeurs manquantes (NaN) et, si c’est le cas, identifie et affiche les colonnes concernées ainsi que leurs premières lignes avec des valeurs manquantes.

In [6]:
if preprocessed_features_all.isna().sum().sum() > 0:
    # Identifier les colonnes avec des valeurs manquantes
    cols_with_na = preprocessed_features_all.columns[preprocessed_features_all.isna().any()].tolist()
    print(f"Columns with missing values: {cols_with_na}")
    
    # Afficher les premières lignes des colonnes concernées
    print(preprocessed_features_all[cols_with_na].head())
else:
    print("No missing values in the dataset")

No missing values in the dataset


# **B. Modèle baseline**

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
X_train_preprocessed.head()

,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_station_bss_id,piezo_bss_code,piezo_measurement_date,piezo_continuity_code,piezo_producer_code,...,piezo_status_Donnée contrôlée niveau 1,piezo_status_Donnée contrôlée niveau 2,piezo_status_Donnée interprétée,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié,piezo_continuity_name_Point lié au point précédent,hydro_qualification_label_Douteuse,hydro_qualification_label_Non qualifiée,hydro_hydro_quantity_elab_QmM
0,226.0,0.003822,0.390918,0.733960,0.462674,1561.0,1563.0,2020-01-01,1.0,2.900484e-12,...,False,True,False,False,False,False,True,False,False,False
1,226.0,0.006802,0.395761,0.706338,0.483565,1466.0,1468.0,2020-01-01,1.0,2.900484e-12,...,False,True,False,False,False,False,True,False,False,True
2,226.0,0.006730,0.386716,0.696756,0.469735,1557.0,1559.0,2020-01-01,1.0,2.900484e-12,...,False,True,False,False,False,False,True,False,False,False
3,226.0,0.006535,0.475977,0.748120,0.501648,1417.0,1419.0,2020-01-01,1.0,2.900484e-12,...,False,True,False,False,False,False,True,True,False,True
4,226.0,0.007127,0.399809,0.703285,0.494880,1416.0,1418.0,2020-01-01,1.0,2.900484e-12,...,False,True,False,False,False,False,True,False,False,False


In [9]:
# Vérification des types de données
pd.set_option('display.max_rows', None)
print(preprocessed_features_all.dtypes)

# Conversion des colonnes datetime
datetime_cols = preprocessed_features_all.select_dtypes(include=['datetime64']).columns
for col in datetime_cols:
    preprocessed_features_all[col] = preprocessed_features_all[col].astype('int64')  # Conversion en timestamp

# Conversion des colonnes booléennes
bool_cols = preprocessed_features_all.select_dtypes(include=['bool']).columns
preprocessed_features_all[bool_cols] = preprocessed_features_all[bool_cols].astype('int')

# Vérification des types après conversion
print(preprocessed_features_all.dtypes)

# Repartir les données entre entraînement et test
X_train_preprocessed = preprocessed_features_all[:len(X_train)]
X_test_preprocessed = preprocessed_features_all[len(X_train):]

piezo_station_update_date                                    float64
piezo_station_investigation_depth                            float64
piezo_station_altitude                                       float64
piezo_station_longitude                                      float64
piezo_station_latitude                                       float64
piezo_station_bss_id                                         float64
piezo_bss_code                                               float64
piezo_measurement_date                                datetime64[ns]
piezo_continuity_code                                        float64
piezo_producer_code                                          float64
meteo_id                                                     float64
meteo_altitude                                               float64
meteo_date                                                   float64
meteo_rain_height                                            float64
meteo_DRR                         

In [10]:
print(len(X_train_preprocessed))
print(len(y_train))

2830316
2830316


In [11]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_preprocessed, y_train)

DecisionTreeClassifier(random_state=42)

In [16]:
from sklearn.metrics import accuracy_score, f1_score

y_test = pd.read_csv("../data/y_test.csv")

accuracy = f1_score(y_test, model.predict(X_test),average='micro')
print(accuracy)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- hydro_hydro_quantity_elab
- hydro_latitude
- hydro_longitude
- hydro_method_label
- hydro_qualification_label
- ...
Feature names seen at fit time, yet now missing:
- hydro_hydro_quantity_elab_QmM
- hydro_qualification_label_Douteuse
- hydro_qualification_label_Non qualifiée
- piezo_continuity_name_Point lié au point précédent
- piezo_obtention_mode_Valeur mesurée
- ...


In [14]:
save_to_csv(model, X_test,mapping_int_to_string,row_index_test,"predictions_decision_tree.csv")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- hydro_hydro_quantity_elab
- hydro_latitude
- hydro_longitude
- hydro_method_label
- hydro_qualification_label
- ...
Feature names seen at fit time, yet now missing:
- hydro_hydro_quantity_elab_QmM
- hydro_qualification_label_Douteuse
- hydro_qualification_label_Non qualifiée
- piezo_continuity_name_Point lié au point précédent
- piezo_obtention_mode_Valeur mesurée
- ...


# **C. Modèle LSTM**

# **D. Tests et validation**